In [ ]:
from bs4 import BeautifulSoup, Tag
import bs4
import requests
import json
from pprint import pprint
from time import sleep
import tldextract
from urllib.parse import urljoin
visited = set()


In [ ]:
def get_html_structure(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    classes = []
    tags = {}
    for tag in soup.find_all():
        if isinstance(tag, Tag):
            
            #check if tag has class
            if tag.has_attr('class'):
                if tag.name not in tags:
                    tags[tag.name] = []
                    
                # only add classes from tags that haven't been added yet to classes
                for c in tag['class']:
                    if c not in classes:
                        classes.append(c)
                        tags[tag.name].append(c)
    return tags

In [ ]:
def save_content(url, content, type='txt'):
    filename = url.replace('/', '_').replace('https:', '').replace('http:', '') + "." + type
    # filename = f"{extracted.domain}_{extracted.suffix}.txt"
    path = "data/faq/" + type + "/" + filename
    print("save_content path: ", path)
    with open(path, 'w', encoding='utf-8') as f:
        f.write(content)

In [ ]:

def process_element(element):
    # Handle string (NavigableString) elements
    if isinstance(element, str) or isinstance(element, bs4.NavigableString):
        return element

    processed_elements = []
    for child in element.children:
        processed_elements.append(process_element(child))

    # Now we handle various tag types
    if element.name == 'a':
        return f'[{"".join(processed_elements)}]({element.get("href", "")})'
    elif element.name == 'b':
        return f'**{"".join(processed_elements)}**'
    elif element.name == 'i':
        return f'_{"".join(processed_elements)}_'
    else:
        return "".join(processed_elements)


In [ ]:
base_url = 'https://faq.leafwell.com/knowledge/'
def valid_url(url):
    # Implement this function to filter out URLs you don't want to visit
    if "facebook" in url:
        return False
    if "twitter" in url:
        return False
    if "instagram" in url:
        return False
    if "mailto" in url:
        return False

    # if an url has "blog" in it, then it is valid
    if "knowledge" in url:
        if not url.startswith('https'):
            url = urljoin(base_url, url)
        return url
    return False
    # return "http" in url

In [ ]:
def get_category_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    menu = soup.find(class_='kb-category-menu')
    links = []
    for link in menu.find_all('a'):
        if link.has_attr('href'):
            if valid_url(link['href']):
                links.append(link['href'])
    return links

In [ ]:
visited = set()
def scrape_faq(url):
    
    url = valid_url(url)
    if url is False:
        return
    if url in visited:
        return
    visited.add(url)
    sleep(2)
    print("scrape Url: ", url,"\n\n")
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    main_content = soup.find(class_='kb-content')

    # Check if main content was found
    if main_content is not None:
        # Get 'kb-article tinymce-content' elements within the main content
        kb_articles = main_content.find_all(class_='kb-article tinymce-content')
        # Check if any 'kb-article tinymce-content' elements were found
        if len(kb_articles) > 0:
            for article in kb_articles:
                # Get a list of dicts of all the links from the article
                # Link dict = {"text_of_link": "url_of_link"}
                Links = []
                links = article.find_all('a')
                # Loop through the links
                for link in links:
                    Links.append(json.dumps({link.get_text(): link.get('href')}))

                # Get the text of the article
                content = " ".join(Links) + "<||>"
                content += article.get_text()
                # Save the content to a file
                save_content(url, content, 'txt')
                save_content(url, process_element(article), 'md')
                # Print the content
                print(content)
                print('----------------------')
        else:
            # find links in the main content
            links = main_content.find_all('a')
            for link in links:
                # Get the link
                link_url = valid_url(link.get('href'))
                # Check if the link is valid
                if link_url:
                    # Scrape the link
                    scrape_faq(link_url)
    

    

In [ ]:
url = 'https://faq.leafwell.com/knowledge/general-information'
links = get_category_links(url)
# for link in links:
#     scrape_faq(link)

In [ ]:
pprint(links)

In [ ]:
scrape_faq('https://faq.leafwell.com/knowledge/general-information')

In [ ]:
faq = 'https://faq.leafwell.com/knowledge/california-state-process'  # replace with your url
# faq_structure = get_html_structure(faq)
scrape_faq(faq)
# print(json.dumps(faq_structure))
# blog = 'https://leafwell.com/blog/is-marijuana-legal-in-bahrain/'
# blog_structure = get_html_structure(blog)
# print(json.dumps(blog_structure))